Name: Sathish Kumar Rajendiran
Task: Week8: 8.3 MongoDB & Tweets Collection
Date: 8/19/2020

In [ ]:
#pip install pymongo library
!pip install pymongo
!pip install tweepy    

In [3]:
#import libraries

# standard library
import os
import sys
from datetime import datetime
import time
import re

# csv, xls, pandas & json
import pandas as pd
import json
import csv
import xlrd

#twitter libraries
import tweepy
from tweepy import StreamListener
from tweepy import Stream
import preprocessor as p
# from tweet-preprocessor import clean,tokenize,parse

#MongoDB libraries
import pymongo
from pymongo import MongoClient

#visualization
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
%matplotlib inline

import timeit
# import random

os.getcwd()

'/Users/sathishrajendiran/ist652-python'

In [4]:
# Connecting to the database
# Connection to Mongo DB
try:
    client = MongoClient('localhost', 27017)
    print ('Authentication OK - You''re now connected to the MongoDB.\n')
# use database named usgs or create it if not there already
    db = client.bball
    # create collection named earthquakes or create it if not there already
    coll = db.bbcoll

#   print('MongoDB: Tweets db with tweets collection has been created')
    print('MongoDB database: ' + str(db))
    print('MongoDB collection:' + str(coll))

except pymongo.errors.ConnectionFailure as e:
    print ('Could not connect to MongoDB: %s' % e )

Authentication OK - Youre now connected to the MongoDB.

MongoDB database: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bball')
MongoDB collection:Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'bball'), 'bbcoll')


In [5]:
#list database defined
client.list_database_names

<bound method MongoClient.list_database_names of MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)>

In [6]:
#search the first item from the collection
coll.find_one()

{'_id': ObjectId('58d992de64a4f3e56d2db3c7'),
 'user': {'profile_background_tile': True,
  'friends_count': 217,
  'profile_sidebar_fill_color': 'EFEFEF',
  'id_str': '513196438',
  'is_translation_enabled': False,
  'profile_link_color': 'F70808',
  'followers_count': 256,
  'location': '',
  'protected': False,
  'default_profile_image': False,
  'contributors_enabled': False,
  'favourites_count': 2187,
  'profile_background_color': 'BFD0D9',
  'statuses_count': 1104,
  'id': 513196438,
  'profile_banner_url': 'https://pbs.twimg.com/profile_banners/513196438/1437359097',
  'created_at': 'Sat Mar 03 13:54:37 +0000 2012',
  'profile_image_url_https': 'https://pbs.twimg.com/profile_images/732514428051181568/OkOVa8Ia_normal.jpg',
  'time_zone': 'Eastern Time (US & Canada)',
  'follow_request_sent': None,
  'listed_count': 1,
  'utc_offset': -14400,
  'lang': 'en',
  'is_translator': False,
  'name': 'Will',
  'description': "Do not worry about tomorrow, for tomorrow will worry about its

In [7]:
docs = coll.find()
print('docs cursor ready!')

docs cursor ready!


In [35]:
# def print_tweet_data(tweets):
#     for tweet in tweets:
#         print('\nDate:', tweet['created_at'])
#         print('From:', tweet['user']['name'])
#         print('Screen Name:', tweet['user']['screen_name'])
#         print('Message', tweet['text'])

In [37]:
# print_tweet_data(doclist[:5])

In [8]:
docs = coll.find()
tw_list =[]
doclist = [tweet for tweet in docs]
for tweet in doclist:
    id = tweet['id']
    user = tweet['user']['name']
    friends_count = tweet['user']['friends_count']
    followers = tweet['user']['followers_count']
    retweets = tweet['retweet_count']
    bg_color = tweet['user']['profile_background_color']
    datets = tweet['created_at']
    text = tweet['text']
    tw_list.append([id,user,friends_count,followers,retweets,bg_color,datets,text])
print('Number of docs: ',len(tw_list))

Number of docs:  2000


In [9]:
#define column names
ColNames = ['id','user','friends_count','followers','retweets','bg_color','datets','text']

# Show all columns and do not truncate in the data frame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

tweetsDF = pd.DataFrame(tw_list,columns=ColNames)

print('Total Number of rows Processed: ',len(tweetsDF))

Total Number of rows Processed:  2000


In [10]:
tweetsDF.head()

,id,user,friends_count,followers,retweets,bg_color,datets,text
0,8.464896e+17,Will,217,256,9739,BFD0D9,Mon Mar 27 22:30:30 +0000 2017,RT @HowardWKYT: The final seconds of the Kentucky-North Carolina game was an emotional roller coaster for Big Blue fans. #marchmadness #WKY…
1,8.464896e+17,Kahlen Donatell,468,386,71,131516,Mon Mar 27 22:30:22 +0000 2017,RT @WhistleSports: When you perfectly time the #UNC buzzer beater 😱🏀 #MarchMadness\n\n(via:@SamuelGrubbs1 ) https://t.co/Ol2ibpZjB4
2,8.464896e+17,Jesús,156,262,9739,C0DEED,Mon Mar 27 22:30:21 +0000 2017,RT @HowardWKYT: The final seconds of the Kentucky-North Carolina game was an emotional roller coaster for Big Blue fans. #marchmadness #WKY…
3,8.464896e+17,"plug,",55,23,13493,000000,Mon Mar 27 22:30:14 +0000 2017,RT @BleacherReport: And then there were four... #MarchMadness https://t.co/0MbxpgAuUC
4,8.464896e+17,Joni Dickerson,381,273,12,EBEBEB,Mon Mar 27 22:30:11 +0000 2017,RT @mycarolinastdnt: RT if you'll be cheering on @GamecockWBB and @dawnstaley tonight. Let's Go Gamecocks! #MarchMadness


In [11]:
##### datetime conversion
tweetsDF['datets'] = tweetsDF['datets'].astype('datetime64[ns]')
tweetsDF['date'] = tweetsDF['datets'].dt.date
tweetsDF['month'] = tweetsDF['datets'].dt.month
tweetsDF['monthday'] = tweetsDF['datets'].dt.day
tweetsDF['weekday'] = tweetsDF['datets'].dt.weekday
tweetsDF['dayname'] = tweetsDF['datets'].dt.day_name()
tweetsDF['monthname'] = tweetsDF['datets'].dt.month_name()
tweetsDF['hour'] = tweetsDF['datets'].dt.hour
tweetsDF['minute'] = tweetsDF['datets'].dt.minute
tweetsDF['secs'] = tweetsDF['datets'].dt.second

In [12]:
#validate Number of retweets by values 
retweets = tweetsDF[tweetsDF['text'].str.startswith('RT')==True]
print('Number of retweets: ',len(retweets))
print('Percentage of retweets {}%'.format(round((len(retweets))/len(tweetsDF['text'])*100,2)))

Number of retweets:  1499
Percentage of retweets 74.95%


In [13]:
#validate Number of retweets by values 
act_tweets = tweetsDF[tweetsDF['text'].str.startswith('RT')==False]
print('Number of actual tweets: ',len(act_tweets))
print('Percentage of actualtweets {}%'.format(round((len(act_tweets))/len(tweetsDF['text'])*100,2)))

Number of actual tweets:  501
Percentage of actualtweets 25.05%


In [14]:
act_tweets

,id,user,friends_count,followers,retweets,bg_color,datets,text,date,month,monthday,weekday,dayname,monthname,hour,minute,secs
7,8.464895e+17,GooglePlayNewsstand,198,1151,0,F5F8FA,2017-03-27 22:30:09,"""It’s OK to believe in the hot hand."" https://t.co/yJOqhlRz8L h/t @ConversationUS #MarchMadness #NCAA",2017-03-27,3,27,0,Monday,March,22,30,9
8,8.464895e+17,2️⃣3️⃣GANG,785,1038,0,FFF04D,2017-03-27 22:30:07,It's gonna be a Civil War for the finals! #MarchMadness,2017-03-27,3,27,0,Monday,March,22,30,7
9,8.464895e+17,allAboutYOU,142,54,0,F5F8FA,2017-03-27 22:30:07,Beach ready? #trend #vip #marketing #marchmadness #UnnecessaryConfessions #younotfromhouston… https://t.co/AtoeIuQuXk,2017-03-27,3,27,0,Monday,March,22,30,7
10,8.464895e+17,IndustryNightInsider,525,2034,0,131516,2017-03-27 22:30:02,Industry Monday! @ShakerChicago #mondaymotivation #internationalwhiskyday #MarchMadness #spanishpaelladay… https://t.co/rTw25qvjRS,2017-03-27,3,27,0,Monday,March,22,30,2
19,8.464895e+17,faith yates,83,98,0,000000,2017-03-27 22:29:50,i won by default 😆 but i'm so proud of @hhn4334 and aiden for our first #marchmadness together https://t.co/XPO3jztjdG,2017-03-27,3,27,0,Monday,March,22,29,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1980,8.464640e+17,alenasbdesign,208,509,0,EBEBEB,2017-03-27 20:48:50,25% OFF SITEWIDE\n#CouponCode: MARCHMADNEZZ\nhttps://t.co/VWjBGlTZfv\n#marchmadness #supersale #onsalenow https://t.co/DDz84pffQy,2017-03-27,3,27,0,Monday,March,20,48,50
1986,8.464640e+17,CollegeBB News,65210,65171,18,C0DEED,2017-03-27 20:48:36,Vegas odds to win the National Championship\n\n1. Gonzaga 3/2\n\n2. North Carolina 7/5\n\n3. Oregon 9/2\n\n4. South Carolina 15/2\n\n#MarchMadness,2017-03-27,3,27,0,Monday,March,20,48,36
1991,8.464639e+17,Steve Clarke,1594,316,0,C0DEED,2017-03-27 20:48:25,@CandaceBurnsTV got the best #marchmadness commercials,2017-03-27,3,27,0,Monday,March,20,48,25
1992,8.464639e+17,Aniket Goel,224,139,0,000000,2017-03-27 20:48:23,How Much Does the NCAA Make off March Madness? | #Investopedia #MarchMadness https://t.co/I8CNi1zLS4,2017-03-27,3,27,0,Monday,March,20,48,23


In [15]:
#number of tweets by date
by_date = act_tweets.groupby(['date']).size().reset_index(name='counts')
by_date = by_date.set_index('date')
by_date.reset_index(level=0,inplace = True,drop=False)
by_date.index += 1
print('Tweets by calender date: ')
by_date

Tweets by calender date: 


,date,counts
1,2017-03-27,501


In [16]:
#number of tweets by hour of day
by_day_hour = act_tweets.groupby(['dayname','hour']).size().reset_index(name='counts')
by_day_hour = by_day_hour.set_index('dayname')
by_day_hour.reset_index(level=0,inplace = True,drop=False)
by_day_hour.index += 1
print('Tweets by hour of the day: ')
by_day_hour

Tweets by hour of the day: 


,dayname,hour,counts
1,Monday,20,55
2,Monday,21,296
3,Monday,22,150
